In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, interact

Parameters:

In [2]:
muX, muY = 4, 4
sigmaX, sigmaY = 3, 3
sigmaD = 0.5  # equivalent to 1/2
dAverage = np.sqrt(muX**2 + muY**2)

2D Gaussian function $f(x, y)$:

In [3]:
def f_xy(x, y):
    return (
        1
        / (2 * np.pi * sigmaX * sigmaY)
        * np.exp(
            -(((x - muX) ** 2) / (2 * sigmaX**2) + ((y - muY) ** 2) / (2 * sigmaY**2))
        )
    )

Distance Gaussian $f_D(d, x, y)$:

In [4]:
def fD(d, x, y):
    return (
        1
        / (np.sqrt(2 * np.pi) * sigmaD)
        * np.exp(-((d - np.sqrt(x**2 + y**2)) ** 2) / (2 * sigmaD**2))
    )

Combined function $f(x, y, d)$:

In [5]:
def f_xy_d(x, y, d):
    return fD(d, x, y) * f_xy(x, y)

Interactive plotting function: one panel for $f(x, y)$, the other for $f(x, y, d)$ with a circle of radius $d$.

In [14]:
def update_plot(d):
    # Create a grid for x and y over the domain [-10, 10]
    x = np.linspace(-10, 10, 200)
    y = np.linspace(-10, 10, 200)
    X, Y = np.meshgrid(x, y)

    img_array_left = f_xy(X, Y)
    img_array_right = f_xy_d(X, Y, d)

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # LEFT PLOT
    # Plot the 2D Gaussian f(x,y)
    axes[0].imshow(
        img_array_left, extent=[-10, 10, -10, 10], origin="lower", aspect="equal"
    )
    axes[0].set_title("Unconditional $f(x, y)$")

    # RIGHT PLOT
    # Plot the combined function f(x,y,d)
    axes[1].imshow(
        img_array_right, extent=[-10, 10, -10, 10], origin="lower", aspect="equal"
    )
    axes[1].set_title(f"Conditional $f(x, y, d={d:.2f})$")

    # Overlay a circle with radius d centered at (0,0)
    circle = plt.Circle((0, 0), d, color="gray", fill=False, lw=1)
    axes[1].add_artist(circle)

    plt.show()

Create the interactive slider for $d$

In [15]:
interact(
    update_plot,
    d=FloatSlider(min=dAverage - 2, max=dAverage + 2, step=0.1, value=dAverage),
)

interactive(children=(FloatSlider(value=5.656854249492381, description='d', max=7.656854249492381, min=3.65685…

<function __main__.update_plot(d)>